<center><h1><strong> Twitter Sentiment Analysis using ML </strong></h1>
<h2> Author: Muhammad Taimoor Khan </h2>

### Importing Dependencies

In [3]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import nltk

In [4]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\PMLS\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [5]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

### Data Preprocessing

In [7]:
twitter_data = pd.read_csv('./data/training.1600000.processed.noemoticon.csv', encoding='ISO-8859-1')
print("Dataset have been read.")

Dataset have been read.


In [8]:
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [9]:
# checking the number of rows and cols
twitter_data.shape

(1599999, 6)

In [11]:
# naming the cols and reading the dataset again
col_names = ['target','id','date','flag','user','text']
twitter_data = pd.read_csv('./data/training.1600000.processed.noemoticon.csv', names=col_names, encoding='ISO-8859-1')
print("Dataset with column names have been read.")

Dataset with column names have been read.


In [12]:
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [13]:
twitter_data.shape

(1600000, 6)

In [14]:
# finding the missing values in the dataset
twitter_data.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

<code> There are no missing values in the dataset. </code>

In [16]:
# finding the distribution of "target" column
twitter_data['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

<code> There are only 2 Labels in "target" column which are 0 and 4 <br> 0 --> Negative Tweet <br> 4 --> Positive Tweet </code>

For our own convenience we can change Label "4" into "1".

In [18]:
twitter_data.replace({'target':{4:1}}, inplace=True)
twitter_data['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

<code> Now, <br> 0 --> Negative Tweet <br> 1 --> Positive Tweet </code>

### Steming

<code>STEMMING:</code> The process of reducing a word to it's root word.

e.g: diving, diver, dived == dive

In [20]:
port_stem = PorterStemmer()
port_stem

<PorterStemmer>

In [21]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]', ' ', content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    
    return stemmed_content

In [22]:
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)

In [23]:
twitter_data.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [24]:
print(twitter_data['stemmed_content'])

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [25]:
print(twitter_data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [26]:
# seperating stemmed data and label
X = twitter_data['stemmed_content'].values
Y = twitter_data['target'].values

In [27]:
print(X)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [28]:
print(Y)

[0 0 0 ... 1 1 1]


### Splitting Data into Training and Testing Data

In [29]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.25, stratify=Y, random_state=2)

In [30]:
print(X.shape,X_train.shape,X_test.shape)

(1600000,) (1200000,) (400000,)


In [31]:
print(X_train)

['staceymonk made suspici two comput share singl internet connect'
 'come rivis cold war sun watchin home away dont love weather'
 'mariedancerr exam next day think stay rebel german im gonna fail tht anyway'
 ... 'school readi new class'
 'look old pictur marcecantu genaro cantu hahaha'
 'love wake folger bad voic deeper']


In [32]:
print(X_test)

['back hurt horribl omg give head ach touch hate cv'
 'hellu everybodi twitterfrend what go day first hav work much work'
 'interest day ahead look like put hardwood floor offic start twitter' ...
 'often rain even day' 'back home feel tire today'
 'well bout get amp head memphi holla yall later bye bye']


As ML algorithms do not recognize textual data, so we have to convert them into numerical data.

In [33]:
vectorizer = TfidfVectorizer()

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [34]:
print(X_train)

  (0, 76623)	0.3079421596528722
  (0, 173944)	0.27361083420633164
  (0, 353480)	0.31024983563960834
  (0, 347166)	0.28801409672291506
  (0, 76067)	0.26933867919175003
  (0, 403997)	0.24502287044549792
  (0, 374356)	0.448146199728837
  (0, 237343)	0.23303345421625404
  (0, 365189)	0.5132080660144132
  (1, 419352)	0.25706702258069447
  (1, 232174)	0.17010366621372047
  (1, 102200)	0.23313119944256036
  (1, 27594)	0.2489748661913732
  (1, 161685)	0.19896977936288707
  (1, 418472)	0.33235992111366175
  (1, 372368)	0.256832440572957
  (1, 417988)	0.3404716584815899
  (1, 74743)	0.26384919818910885
  (1, 327520)	0.5889858120350424
  (1, 75463)	0.20046450884815895
  (2, 19892)	0.25851264547753383
  (2, 390630)	0.32485953666504225
  (2, 122157)	0.2567074056592465
  (2, 144467)	0.20332052692124486
  (2, 170687)	0.17505789138153294
  :	:
  (1199995, 441344)	0.38423553348045847
  (1199995, 380922)	0.33412704097576246
  (1199995, 28119)	0.34317222539429654
  (1199995, 120816)	0.3438709016302232
  

In [35]:
print(X_test)

  (0, 396888)	0.3522679708361543
  (0, 288731)	0.279794468955125
  (0, 165295)	0.27283113951697135
  (0, 162747)	0.34342514882547964
  (0, 155084)	0.26850580709852695
  (0, 153988)	0.24966370327216753
  (0, 142254)	0.2816038871445881
  (0, 83586)	0.46809413234577185
  (0, 29794)	0.2065818431550562
  (0, 2660)	0.3601783047868634
  (1, 427020)	0.34364342763470335
  (1, 421479)	0.3188692473686777
  (1, 267393)	0.20801913348714463
  (1, 157078)	0.6215471764912583
  (1, 154194)	0.3619263202636623
  (1, 143622)	0.1524821950529528
  (1, 127362)	0.241680550521661
  (1, 119508)	0.3338431294300748
  (1, 89868)	0.1629004007027813
  (2, 403364)	0.2117688080216412
  (2, 366053)	0.23383296032169892
  (2, 313254)	0.26827307453006766
  (2, 286263)	0.29500402660299
  (2, 230724)	0.2079697128335293
  (2, 225144)	0.1760051569672463
  :	:
  (399996, 394299)	0.1918649747558794
  (399996, 267912)	0.3309813811971613
  (399996, 241630)	0.3775126043628339
  (399996, 144467)	0.24081453257477542
  (399996, 61399

### Training The ML Model

### Logistic Regression

In [36]:
model = LogisticRegression(max_iter=1000)

In [37]:
model.fit(X_train,Y_train)

LogisticRegression(max_iter=1000)

### ML Model Evaluation

#### Accuracy Score

In [ ]:
X_train_predictions = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train, X_train_predictions)

0.8104558333333334

In [ ]:
print(f"Accuracy on the training data is {training_data_accuracy}")

Accuracy on the training data is 0.8104558333333334


In [41]:
X_test_predictions = model.predict(X_test)
testing_data_accuracy = accuracy_score(Y_test, X_test_predictions)

In [42]:
print(f"Accuracy on the testing data is {testing_data_accuracy}")


Accuracy on the testing data is 0.77805


#### Saving The Trained Model

In [43]:
import pickle

In [44]:
filename = 'trained_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
filename_vectorizer = 'vectorizer.sav'
pickle.dump(vectorizer, open(filename_vectorizer, 'wb'))

#### Using The Saved Model For Future Predictions

In [45]:
# loading the saved model
loaded_model = pickle.load(open('./trained_model.sav', 'rb'))

In [53]:
X_new = X_test[200]
print(Y_test[200])
prediction = model.predict(X_new)
if (prediction[0] == 0):
    print("It is a negative tweet.")
    
else:
    print("It is a positive tweet.")

1
It is a positive tweet.
